In [1]:
from re import sub, findall, MULTILINE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from unicodedata import normalize
from json import loads
from csv import reader,writer,QUOTE_ALL
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import FreqDist, pos_tag
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from pickle import load
from operator import itemgetter
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier


In [2]:
attr_description = {
    0: 'conjunction, coordinating', 1: 'numeral, cardinal',
    2: 'determiner', 3: 'foreign word', 4: 'preposition or conjunction, subordinating',
    5: 'adjective or numeral, ordinal', 6: 'adjective, comparative',
    7: 'adjective, superlative', 8: 'modal auxiliary',                    
    9: 'noun, common, singular or mass', 10: 'noun, proper, singular',
    11: 'noun, proper, plural', 12: 'noun, common, plural', 13:'pre-determiner',
    14: 'genitive marker', 15: 'pronoun, personal', 16: 'pronoun, possessive',
    17: 'adverb', 18: 'adverb, comparative', 19: 'adverb, superlative',
    20: 'particle', 21: '"to" as preposition/infinitive',
    22: 'interjection', 23: 'verb, base form', 24: 'verb, past tense',
    25: 'verb, present participle or gerund', 26: 'verb, past participle',
    27: 'verb, present tense, not 3rd person singular',
    28: 'verb, present tense, 3rd person singular', 29: 'WH-determiner', 30: 'WH-pronoun',
    31: 'WH-pronoun, possessive',32: 'Wh-adverb',33:'number of words', 34:'number of words per sentence',
    35: 'number of captalized words',36: 'percentage of stopwords', 37:'number of punctuation',
    38:'number of quotes', 39:'number of URls'    
}

In [8]:
def remove_ulr_corpus(text):
    new_text = sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', text, flags=MULTILINE)
    count_matches = len(findall(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',text))
    return new_text, count_matches

def remove_stopwords_corpus(text):
    content = []
    for w in text:
        if w.lower().strip() not in stopwords.words('english'):
            content.append(w.lower().strip())

    return content

def remove_mention_corpus(text):
    new_text = sub(r'@\w+ ?','',text)
    count_matches = len(findall(r'@\w+ ?',text))
    
    return new_text, count_matches

def tokenizer(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    return tokens

#"WP$":0,


#"WP$":0,
#'WH-pronoun, possessive' 31 WP$
#'interjection’  22 UH
#'genitive marker' 14
#'pre-determiner'   13
#‘adverb, superlative'.   19
#'foreign word'    3

# 0: 'conjunction, coordinating', 1: 'numeral, cardinal',
#    2: 'determiner', 3: 'foreign word', 4: 'preposition or conjunction, subordinating',
#    5: 'adjective or numeral, ordinal', 6: 'adjective, comparative',
#    7: 'adjective, superlative', 8: 'modal auxiliary',                    
#    9: 'noun, common, singular or mass', 10: 'noun, proper, singular',
##    11: 'noun, proper, plural', 12: 'noun, common, plural', 13:'pre-determiner',
#    14: 'genitive marker', 15: 'pronoun, personal', 16: 'pronoun, possessive',
#    17: 'adverb', 18: 'adverb, comparative', 19: 'adverb, superlative',
#    20: 'particle', 21: '"to" as preposition/infinitive',
#    22: 'interjection', 23: 'verb, base form', 24: 'verb, past tense',
#    25: 'verb, present participle or gerund', 26: 'verb, past participle',
 #   27: 'verb, present tense, not 3rd person singular',
 #   28: 'verb, present tense, 3rd person singular', 29: 'WH-determiner', 30: 'WH-pronoun',
 #   31: 'WH-pronoun, possessive',32: 'Wh-adverb',33:'number of words', 34:'number of words per sentence',
 #   35: 'number of captalized words',36: 'percentage of stopwords', 37:'number of punctuation',
 #   38:'number of quotes', 39: 'author',40:'basic_tags', 41:'formatting_tags', 42:'forms_inputs_tags',
 #   43:'frames_tags', 44:'images_tags', 45:'audio_video_tags', 46:'links_tags',
 #   47:'lists_tags', 48:'tables_tags', 49:'semantics_tags', 50:'metainfo_tags',
 #   51:'programming_tags', 52:'advertisement'
            
  #  word_tagged_dict = {"CC":0, "CD":0, "DT":0, "FW****":0, "IN":0, 
                      #  "JJ":0, "JJR":0,"JJS":0, "MD":0, "NN":0,
                       # "NNP":0, "NNPS":0, "NNS":0,"PDT*****":0, "POS****":0, 
                        #"PRP":0, "PRP$":0, "RB****":0, "RBR":0, "RBS***":0,
                       # "RP":0, "TO":0, "UH****":0, "VB":0, "VBD":0, 
                       # "VBG":0, "VBN":0,"VBP":0, "VBZ":0, "WDT":0,
                        #"WP":0, "WRB":0}            
def part_of_speech_tagger(tokens):
    ''' 
    More details in nltk.help.upenn_tagset()
        CC, CD, DT, FW, IN, JJ, JJR, JJS, LS, MD, NN, NNP,
        NNPS, NNS, PDT, POS, PRP, PRP$, RB, RBR, RBS, RP, SYM,
        TO, UH, VB, VBD, VBG, VBN, VBP, VBZ, WDT, WP, WP$, WRB
    '''
    word_tagged_dict = {"CC":0, "CD":0, "DT":0, "FW":0, "IN":0, 
                            "JJ":0, "JJR":0,"JJS":0, "MD":0, "NN":0,
                            "NNP":0, "NNPS":0, "NNS":0,"PDT":0, "POS":0, 
                            "PRP":0, "PRP$":0, "RB":0, "RBR":0, "RBS":0,
                            "RP":0, "TO":0, "UH":0, "VB":0, "VBD":0, 
                            "VBG":0, "VBN":0,"VBP":0, "VBZ":0, "WDT":0,
                            "WP":0, "WP$":0, "WRB":0} 

    word_tagged_list, result = pos_tag(tokens), []
    for item in  word_tagged_list:
        if(item[1] in word_tagged_dict.keys()):
            word_tagged_dict[item[1]] += 1

    word_tagged_sorted = sorted(word_tagged_dict.items(), key=itemgetter(0))
    word_tagged_list = []
    for i in word_tagged_sorted:
        word_tagged_list.append(i[1])
    #word_tagged_list.pop(10)
    #word_tagged_list.pop(30)
    return word_tagged_list

def count_captalized_words(tokens):
    nro_upper_words = 0
    for i in tokens:
        if(i.isupper()):
            nro_upper_words += 1
    return nro_upper_words

def count_per_stopwords(tokens):
    total_words, words_per_sent = len(tokens), 0.0
    stopwords_nltk = stopwords.words('english')
    count = 0
    for w in tokens:
        if w.lower().strip() in stopwords_nltk:
            count += 1
    if(total_words > 0):
        words_per_sent = round(count/total_words,2)
    return words_per_sent

def count_punctuation(text, regular_exp):
    #r'[' + punctuation+ r']+' and r'["\']+'
    count = 0
    tokenizer = RegexpTokenizer(regular_exp)
    tokens = tokenizer.tokenize(text)
    count = len(tokens)

    return count

def count_quotes(text):
    count = 0
    tokenizer = RegexpTokenizer(r'["\']+')
    tokens = tokenizer.tokenize(text)
    count = len(tokens)

    return count

def sentence_level_attr(text, tokens):
    word_count, word_sent = 0, 0
    words_per_sent = 0.0

    #word count of the whole text
    vocabulary = FreqDist(tokens)
    word_count = len(vocabulary.keys())

    #word count by setence
    sent_tokenize_list = sent_tokenize(text)
    for i in sent_tokenize_list:
        tkn = tokenizer(i)
        vocabulary = FreqDist(tkn)
        word_sent += len(vocabulary.keys())

    if(len(sent_tokenize_list) > 0):
        words_per_sent = round(word_sent/len(sent_tokenize_list), 2)

    return word_count, words_per_sent

In [9]:
def calculate_entropy(X):
    lines = np.shape(X)[0]
    columns = np.shape(X)[1]    
    result = {}
    
    for i in range(columns):
        
        list_probabilities = []
        sum_values,entropy  = 0.0, 0.0
        for j in range(lines):
            sum_values += X[j][i]
        for j in range(lines):
            if(sum_values > 0):
                list_probabilities.append(X[j][i]/sum_values)
            else:
                list_probabilities.append(0.0)
        
        for pi in list_probabilities:
            if pi > 0:
                entropy +=  pi * log(pi, 2)
                
        if(entropy != 0.0):
            entropy = entropy/log(lines,2)*(-1)
        else:
            entropy = 0.0
        
        result[i] = entropy
            
    return result

def plot_entropy_bar_chart(entropy_attr_values, range_ranking):

    top_attr = sorted(entropy_attr_values.items(), key=itemgetter(1),reverse=True)[:range_ranking]
    
    x,y = [],[]
    for i in top_attr:
        #print(attr_description[i[0]],'=>\t',i[1])
        x.append(attr_description[i[0]])
        y.append(i[1])
        
    plt.figure(figsize=(10,15))
    plt.title("Entropy Features Values")

    plt.barh(range(len(top_attr)), y,
           color="r", align="center")
    plt.xticks(fontsize=12)
    plt.yticks(range(len(top_attr)), x,fontsize=15)
    #plt.ylim([-1, top_10_attr])
    plt.show()        

In [10]:
def statistics_metric_value(metric_list):
    mean_list = [0,0]
    elements_class = []
    z_critical = stats.norm.ppf(q = 0.975) #1.959963984540054
    result = {0:[], 1:[]}
    for i in range(0,len(mean_list)):
        for j in metric_list:
            mean_list[i] += j[i]
            elements_class.append(j[i])

        standard_dev = statistics.stdev(elements_class) 
        margin_of_error = z_critical * (standard_dev/sqrt(len(elements_class)))
        confidence_interval = "{:4.2f}".format(margin_of_error)
        
        mean_list[i] = mean_list[i]/len(metric_list)

        #result[i] = ["{:2.4f}".format(mean_list[i]), confidence_interval]
        result[i] = "{:2.4f}".format(mean_list[i])
    
    return result

def calculate_mean_confmatrix(confusion_list):
    result = [[0.0,0.0],[0.0,0.0]]
    for cm in confusion_list:
        for i in range(0,len(result)):
            for j in range(0,len(result)):
                result[i][j] += cm[i][j]
    for i in range(0,len(result)):
        for j in range(0,len(result)):
            result[i][j] = result[i][j]/len(confusion_list)
    return result

def show_confusion_matrix(cm):
    for i in cm:
        a = "{:.2f}".format(i[0])
        b = "{:.2f}".format(i[1])
        print("["+a+" "+b+"]")


In [11]:
# Testing
text, count_urls = remove_ulr_corpus("Hello home how are you!!!,,, - dd")
tokens = tokenizer(text)

pos_tag_numbers   = part_of_speech_tagger(tokens)
word_count, words_per_sent = sentence_level_attr(text,tokens)
nr_captalized_words  = count_captalized_words(tokens)
nr_per_stopwords = count_per_stopwords(tokens)
nr_punctuation = count_punctuation(text,r'['+punctuation+r']+')
nr_quotes = count_punctuation(text,r'["\']+')